<a href="https://colab.research.google.com/github/lalitmendhule/Deep-Learning/blob/main/Prac5_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install TensorFlow (if not already installed in Colab)
!pip install tensorflow

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout
from tensorflow.keras.utils import to_categorical


In [ ]:
file_path = "/content/process_data.csv"  # In Colab, put your dataset here
data = pd.read_csv(file_path)
print(data.head())
print(data["tissue"].value_counts())


       Gene     Sample  FPKM description        tissue metastasis
0    TSPAN6  CA.102548  0.93   CA.102548  breast tumor        yes
1      TNMD  CA.102548  0.00   CA.102548  breast tumor        yes
2      DPM1  CA.102548  0.00   CA.102548  breast tumor        yes
3     SCYL3  CA.102548  5.78   CA.102548  breast tumor        yes
4  C1orf112  CA.102548  2.83   CA.102548  breast tumor        yes
tissue
breast tumor            607380
normal breast tissue    103887
normal br                    1
Name: count, dtype: int64


In [ ]:
# Select features and labels
X = data.pivot(index="Sample", columns="Gene", values="FPKM").fillna(0)
y = data.groupby("Sample")["tissue"].first()

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # 0 = normal, 1 = breast tumor

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape for RNN [samples, timesteps, features]
X_reshaped = np.expand_dims(X_scaled, axis=-1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)


In [ ]:
model = Sequential()
model.add(SimpleRNN(64, input_shape=(X_train.shape[1], 1), activation='tanh'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 64)             │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,337 (24.75 KB)

 Trainable params: 6,337 (24.75 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    verbose=1
)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.4091 - loss: 0.7574 - val_accuracy: 0.3333 - val_loss: 0.8096
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5000 - loss: 0.7177 - val_accuracy: 0.3333 - val_loss: 0.7857
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.6818 - loss: 0.6624 - val_accuracy: 0.3333 - val_loss: 0.7596
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.7273 - loss: 0.6354 - val_accuracy: 0.3333 - val_loss: 0.7326
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.6818 - loss: 0.6119 - val_accuracy: 0.6667 - val_loss: 0.7034
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.7273 - loss: 0.5786 - val_accuracy: 0.6667 - val_loss: 0.6756
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.6364 - loss: 0.5753 - val_accuracy: 0.6667 - val_loss: 0.6515
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.7727 - loss: 0.5456 - val_accuracy: 0.6667 - val_loss: 0.6263
Epoch 9/10
1/1 ━

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.2f}")

# Predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")


Test Accuracy: 0.88
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 765ms/step


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Confusion Matrix:
 [[6 1]
 [0 1]]

Classification Report:
                       precision    recall  f1-score   support

        breast tumor       1.00      0.86      0.92         7
normal breast tissue       0.50      1.00      0.67         1

            accuracy                           0.88         8
           macro avg       0.75      0.93      0.79         8
        weighted avg       0.94      0.88      0.89         8

